## Implementazione di un modello per la realizzazione di Burn Severity Map

<b>Radars and Remote Sensing</b> a.a. 2022-23

<b>Studente:</b> Stefano Hinic, stefano.hinic@student.univaq.it

<b>Università degli Studi dell'Aquila</b>


## 0.1) Import delle librerie
Import della librerie:
<ol>
  <li><b>ee</b>: Earth Engine Python API</li>
  <li><b>geemap</b>: Package per analisi geospaziale e visualizzazine con Google Earth Engine (GEE)</li>
</ol>

In [ ]:
import ee
import geemap

## 0.2) Autenticazione
Prima di utilizzare la libreria client Python di Earth Engine, è necessario autenticarsi e utilizzare le credenziali Google per inizializzare il client Python.

In [ ]:
ee.Authenticate()

## 0.3) Inizializzazione
Il passaggio di inizializzazione verifica che siano state create credenziali valide e popola la libreria client Python con i metodi supportati dal server back-end.

In [ ]:
ee.Initialize()

## 1) Selezionare l’Area of Interest (AOI)

In [ ]:
#Definisco l'AOI attraverso le coordinate del poligono che racchiude l'area di studio
AOI = ee.Geometry.Polygon(
    [
        [14.029885, 42.053813],
        [13.996247, 42.047695],
        [13.947505, 42.06248],
        [13.898076, 42.101721],
        [13.893271, 42.125152],
        [13.93858, 42.144501],
        [13.985263, 42.131263],
        [14.029199, 42.077262],
        [14.029885, 42.053813]
    ])

## 2) Selezionare un time frame pre-incendio e post-incendio, per gli eventi del 2017 e del 2023


In [ ]:
#Parametri pre-fire 2017
start_prefire_2017 = ee.Date('2017-07-20')
finish_prefire_2017 = ee.Date('2017-08-18')

#Parametri post-fire 2017
start_endfire_2017 = ee.Date('2017-09-02')
finish_endfire_2017 = ee.Date('2017-09-30')

#Parametri pre-fire 2023
start_prefire_2023 = ee.Date('2023-07-01')
finish_prefire_2023 = ee.Date('2023-07-24')

#Parametri post-fire 2023
start_endfire_2023 = ee.Date('2023-08-01')
finish_endfire_2023 = ee.Date('2023-08-30')


## 3) Eseguire un filtraggio per ottenere granuli meno nuvolosi

In [ ]:
# 3 - Creo una ImageCollection con i dati di Sentinel 2 (S2) per pre-incendio e post-incendio 2017
collection_prefire_2017 = ee.ImageCollection("COPERNICUS/S2_SR") \
.filterDate(start_prefire_2017, finish_prefire_2017) \
.filterBounds(AOI) \
.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',1)); 
#print(collection_prefire, 'Image Collection')

collection_endfire_2017 = ee.ImageCollection("COPERNICUS/S2_SR") \
.filterDate(start_endfire_2017, finish_endfire_2017) \
.filterBounds(AOI) \
.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',1)); 
#print(collection_endfire, 'Image Collection')

# 3 - Creo una ImageCollection con i dati di Sentinel 2 (S2) per pre-incendio e post-incendio 2023
collection_prefire_2023 = ee.ImageCollection("COPERNICUS/S2_SR") \
.filterDate(start_prefire_2023, finish_prefire_2023) \
.filterBounds(AOI) \
.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',1)); 
#print(collection_prefire, 'Image Collection')

collection_endfire_2023 = ee.ImageCollection("COPERNICUS/S2_SR") \
.filterDate(start_endfire_2023, finish_endfire_2023) \
.filterBounds(AOI) \
.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',1)); 
#print(collection_endfire, 'Image Collection')


In [ ]:
"""
#debug
print(len(collection_endfire_2023.getInfo()['features']))
for immagine in collection_endfire_2023.getInfo()['features']:
    print(immagine['id'], immagine['properties']['CLOUDY_PIXEL_PERCENTAGE'])
#collection_endfire_2017.getInfo()['features'][0]['properties']['CLOUDY_PIXEL_PERCENTAGE']
"""

## 4) Calolo di una statistica di tutti i passaggi (come mediana o media)

In [ ]:
#Prendo la mediana delle variabili ImageCollection collection_prefire_20** e collection_endfire_20**. Essa rappresenta una 
#statistica di tutti i passaggi. Al termine di questa operazione ottengo le immagini image_prefire_20** e image_endfire_**
#NB: avrei potuto anche prendere direttamente una sola immagine, non nuvolosa, senza passare per la statistica sei passaggi 

#Incendio 2017
image_prefire_2017 = ee.Image(collection_prefire_2017.median())
image_endfire_2017 = ee.Image(collection_endfire_2017.median())

#Incendio 2023
image_prefire_2023 = ee.Image(collection_prefire_2023.median())
image_endfire_2023 = ee.Image(collection_endfire_2023.median())

## 5) Calcolare il NBR pre-incendio e post-incendio per gli eventi del 2017 e del 2023


In [ ]:
#Calcolo del Normalized Burn Ratio (NBR) pre e post incendio 2017
preNBR_2017 = image_prefire_2017.normalizedDifference(['B8', 'B12'])
endNBR_2017 = image_endfire_2017.normalizedDifference(['B8', 'B12'])

#Calcolo del Normalized Burn Ratio (NBR) pre e post incendio 2023
preNBR_2023 = image_prefire_2023.normalizedDifference(['B8', 'B12'])
endNBR_2023 = image_endfire_2023.normalizedDifference(['B8', 'B12'])

## 6) Calcolare il dNBR per gli eventi del 2017 e del 2023, e per il periodo intercorso tra i due incendi

In [ ]:
#Calcolo il differenced NBR (dNBR) per l'incendio del 2017
#dNBR = PrefireNBR - EndfireNBR
dNBR_unscaled_2017 = preNBR_2017.subtract(endNBR_2017)
#Lo scalo per poi riferirmi agli standard USGS
dNBR_2017 = dNBR_unscaled_2017.multiply(1000)

#Calcolo il differenced NBR (dNBR) per l'incendio del 2023
#dNBR = PrefireNBR - EndfireNBR
dNBR_unscaled_2023 = preNBR_2023.subtract(endNBR_2023)
#Lo scalo per poi riferirmi agli standard USGS
dNBR_2023 = dNBR_unscaled_2023.multiply(1000)

#Calcolo il differenced NBR (dNBR) per il periodo tra i due incendi
#dNBR = endfireNBR_2017 - prefireNBR2023
dNBR_unscaled_between = endNBR_2017.subtract(preNBR_2023)
#Lo scalo per poi riferirmi agli standard USGS
dNBR_between = dNBR_unscaled_between.multiply(1000)


## 7) Classificare i dNBR in classi di severità, con riferimento allo standard USGS

In [ ]:
#Definisco un SLD style composto da intervalli discreti da applicare all'immagine, in modo da utilizzare lo standard USGS
sld_intervals = '<RasterSymbolizer>' + \
    '<ColorMap type="intervals" extended="False" >' + \
      '<ColorMapEntry color="#ffffff" quantity="-1000" label="NA"/>' + \
      '<ColorMapEntry color="#7a8737" quantity="-250" label="Enhanced Regrowth, High" />' + \
      '<ColorMapEntry color="#acbe4d" quantity="-100" label="Enhanced Regrowth, Low" />' + \
      '<ColorMapEntry color="#0ae042" quantity="100" label="Unburned" />' + \
      '<ColorMapEntry color="#fff70b" quantity="270" label="Low Severity" />' + \
      '<ColorMapEntry color="#ffaf38" quantity="440" label="Moderate-low Severity" />' + \
      '<ColorMapEntry color="#ff641b" quantity="660" label="Moderate-high Severity" />' + \
      '<ColorMapEntry color="#a41fd6" quantity="1300" label="High Severity" />' + \
    '</ColorMap>' + \
  '</RasterSymbolizer>'

## 8) Plottare ed analizzare i risultati ottenuti

## 8.1) Creazione e plot di una mappa interattiva, centrata sull'AOI, con lo zoom selezionato

In [ ]:
centro=[42.08853000000005,13.922270000000026]
mappa1=geemap.Map(center=centro, zoom=12)
mappa1.add_basemap("Satellite")
mappa1

## 8.2) Plot della statistica di tutti i passaggi, pre e post incendio, sia del 2017 che del 2023

In [ ]:
s2rgb = {
'min': 0,
'max': 2500,
'gamma': 0.89,
'bands': ['B4', 'B3', 'B2']}

mappa1.addLayer(image_prefire_2017.clip(AOI), s2rgb, "Immagine prima dell'incendio 2017" )
mappa1.addLayer(image_endfire_2017, s2rgb, "Immagine dopo l'incendio 2017")
mappa1.addLayer(image_prefire_2023, s2rgb, "Immagine prima dell'incendio 2023" )
mappa1.addLayer(image_endfire_2023, s2rgb, "Immagine dopo l'incendio 2023")
mappa1

## 8.3a) Plot dell'immagine meno nuvolosa, pre e post incendio, sia del 2017 che del 2023

In [ ]:
#Acquisizione dell'immagine con la proprietà "CLOUDY_PIXEL_PERCENTAGE" minore, pre e post incendio, sia del 2017 che del 2023
single_image_prefire_2017=collection_prefire_2017.getInfo()['features'][0]
single_image_endfire_2017=collection_endfire_2017.getInfo()['features'][1]
single_image_prefire_2023=collection_prefire_2023.getInfo()['features'][0]
single_image_endfire_2023=collection_endfire_2023.getInfo()['features'][0]
data_prefire_2017=""
data_endfire_2017=""
data_prefire_2023=""
data_endfire_2023=""
#Incendio 2017
for immagine in collection_prefire_2017.getInfo()['features']:
    if (immagine['id'][-2:-1]!="U" and immagine['properties']['CLOUDY_PIXEL_PERCENTAGE']<single_image_prefire_2017['properties']['CLOUDY_PIXEL_PERCENTAGE']):
        single_image_prefire_2017=immagine
    data=single_image_prefire_2017["id"].split("/")[2][:8]
    anno=data[:4]
    mese=data[4:6]
    giorno=data[6:8]
    data_prefire_2017=giorno+"/"+mese+"/"+anno

for immagine in collection_endfire_2017.getInfo()['features']:
    if (immagine['id'][-2:-1]!="U" and immagine['properties']['CLOUDY_PIXEL_PERCENTAGE']<single_image_endfire_2017['properties']['CLOUDY_PIXEL_PERCENTAGE']):
        single_image_endfire_2017=immagine
    data=single_image_endfire_2017["id"].split("/")[2][:8]
    anno=data[:4]
    mese=data[4:6]
    giorno=data[6:8]
    data_endfire_2017=giorno+"/"+mese+"/"+anno

#Incendio 2023
for immagine in collection_prefire_2023.getInfo()['features']:
    if (immagine['id'][-2:-1]!="U" and immagine['properties']['CLOUDY_PIXEL_PERCENTAGE']<single_image_prefire_2023['properties']['CLOUDY_PIXEL_PERCENTAGE']):
        single_image_prefire_2023=immagine
    data=single_image_prefire_2023["id"].split("/")[2][:8]
    anno=data[:4]
    mese=data[4:6]
    giorno=data[6:8]
    data_prefire_2023=giorno+"/"+mese+"/"+anno


for immagine in collection_endfire_2023.getInfo()['features']:
    if (immagine['id'][-2:-1]!="U" and immagine['properties']['CLOUDY_PIXEL_PERCENTAGE']<single_image_endfire_2023['properties']['CLOUDY_PIXEL_PERCENTAGE']):
        single_image_endfire_2023=immagine
    data=single_image_endfire_2023["id"].split("/")[2][:8]
    anno=data[:4]
    mese=data[4:6]
    giorno=data[6:8]
    data_endfire_2023=giorno+"/"+mese+"/"+anno
    
#Creo le immagini con 'CLOUDY_PIXEL_PERCENTAGE' minore, sfruttando l'id
single_image_prefire_2017=ee.Image(single_image_prefire_2017['id'])
single_image_endfire_2017=ee.Image(single_image_endfire_2017['id'])
single_image_prefire_2023=ee.Image(single_image_prefire_2023['id'])
single_image_endfire_2023=ee.Image(single_image_endfire_2023['id'])

#Definisco una nuova mappa interattiva e plotto le immagini
mappa2=geemap.Map(center=centro, zoom=12)
mappa2.add_basemap("Satellite")
mappa2.addLayer(single_image_prefire_2017, s2rgb, "Immagine prima dell'incendio. Data: " + data_prefire_2017)
mappa2.addLayer(single_image_endfire_2017, s2rgb, "Immagine dopo l'incendio. Data: " + data_endfire_2017)
mappa2.addLayer(single_image_prefire_2023, s2rgb, "Immagine prima dell'incendio. Data: " + data_prefire_2023)
mappa2.addLayer(single_image_endfire_2023, s2rgb, "Immagine dopo l'incendio. Data: " + data_endfire_2023)
mappa2

## 8.3b) Confronto immagini RGB pre e post-incendio del 2017

In [ ]:
mappa2b = geemap.Map()
mappa2b.addLayer
mappa2b.addLayer
left_layer = geemap.ee_tile_layer(single_image_prefire_2017, s2rgb, "Immagine prima dell'incendio. Data: " + data_prefire_2017)
right_layer = geemap.ee_tile_layer(single_image_endfire_2017, s2rgb, "Immagine dopo l'incendio. Data: " + data_endfire_2017)

mappa2b = geemap.Map(center=centro, zoom=13)
mappa2b.split_map(left_layer, right_layer)
mappa2b

## 8.4) Plot dell'NBR pre e post incendio, sia del 2017 che del 2023

In [ ]:
#Definisco una nuova mappa interattiva e plotto le immagini
mappa3=geemap.Map(center=centro, zoom=12)
mappa3.add_basemap("Satellite")
grey = ['white', 'black']
mappa3.addLayer(preNBR_2017.clip(AOI), {'min': -1, 'max': 1, 'palette': grey}, 'Prefire 2017 NBR')
mappa3.addLayer(endNBR_2017.clip(AOI), {'min': -1, 'max': 1, 'palette': grey}, 'Postfire 2017 NBR')
mappa3.addLayer(preNBR_2023.clip(AOI), {'min': -1, 'max': 1, 'palette': grey}, 'Prefire 2023 NBR')
mappa3.addLayer(endNBR_2023.clip(AOI), {'min': -1, 'max': 1, 'palette': grey}, 'Postfire 2023NBR')
mappa3

## 8.5) Plot del dNBR per gli incendi del 2017 e del 2023, e per il periodo intercorso

In [ ]:
#Definisco una nuova mappa interattiva e plotto le immagini
mappa4=geemap.Map(center=centro, zoom=12)
mappa4.add_basemap("Satellite")
grey = ['white', 'black']
mappa4.addLayer(dNBR_2017.clip(AOI), {'min': -1000, 'max': 1000, 'palette': grey}, 'dNBR 2017 B&W', 0)
mappa4.addLayer(dNBR_2023.clip(AOI), {'min': -1000, 'max': 1000, 'palette': grey}, 'NBR 2023 B&W', 0)
mappa4.addLayer(dNBR_between.clip(AOI), {'min': -1000, 'max': 1000, 'palette': grey}, 'dNBR tra gli incendi B&W', 0)
mappa4

## 8.6) Plot del dNBR per gli incendi del 2017 e del 2023, e per il periodo intercorso, classificati secondo lo standard USGS

In [ ]:
#Definisco la legenda secondo gli standard USGS
legend_dict = {
    "Enhanced Regrowth, High": "7a8737",
    "Enhanced Regrowth, Low": "acbe4d",
    "Unburned": "0ae042",
    "Low Severity": "fff70b",
    "Moderate-low Severity": "ffaf38",
    "Moderate-high Severity": "ff641b",
    "High Severity": "a41fd6",
    "NA": "ffffff",
}

geemap.create_legend(
    title='dNBR standard USGS',
    legend_dict=legend_dict,
    draggable=False,
    output='dNBR_USGS_legend.html',
)
style = {
    'position': 'fixed',
    'z-index': '9999',
    'border': '2px solid grey',
    'background-color': 'rgba(255, 255, 255, 0.8)',
    'border-radius': '10px',
    'padding': '5px',
    'font-size': '14px',
    'bottom': '20px',
    'right': '5px',
}

#Definisco una nuova mappa interattiva, plotto le immagini e aggiungo la legenda
mappa5=geemap.Map(center=centro, zoom=12)
mappa5.add_basemap("Satellite")
#Aggiungo l'immagine come layer caratterizzata dai valori di dNBR classificati secondo lo standard USGS, per l'evento del 2017
mappa5.addLayer(dNBR_2017.sldStyle(sld_intervals).clip(AOI), {}, "dNBR  2017 classificato")
#Aggiungo l'immagine come layer caratterizzata dai valori di dNBR classificati secondo lo standard USGS, per l'evento del 2023
mappa5.addLayer(dNBR_2023.sldStyle(sld_intervals).clip(AOI), {}, "dNBR  2023 classificato")
#Aggiungo l'immagine come layer caratterizzata dai valori di dNBR classificati secondo lo standard USGS, tra i due eventi
mappa5.addLayer(dNBR_between.sldStyle(sld_intervals).clip(AOI), {}, "dNBR  between classificato")
#Aggiungo la legenda
mappa5.add_legend(
    title='dNBR standard USGS', legend_dict=legend_dict, draggable=False, style=style
)
mappa5

## Oss: Anche lievi cambiamenti nella vegetazione possono essere rilevati come bruciature "Low Severity", motivo per cui questa classe deve essere trattata con particolare attenzione.

## 8.7) Confronto dei dNBR per gli incendi del 2017 e del 2023

In [ ]:
#Definisco il layer sulla sx della sbarra verticale e quello sulla dx
left_layer = geemap.ee_tile_layer(dNBR_2023.sldStyle(sld_intervals).clip(AOI), {}, "dNBR 2023 classificato USGS")
right_layer = geemap.ee_tile_layer(dNBR_2017.sldStyle(sld_intervals).clip(AOI), {}, "dNBR 2017 classificato USGS")

#Definisco una nuova mappa interattiva, plotto le immagini e aggiungo la legenda
mappa6 = geemap.Map(center=centro, zoom=12)
mappa6.split_map(left_layer, right_layer)
mappa6.add_legend(
    title='dNBR standard USGS', legend_dict=legend_dict, draggable=False, style=style
)
mappa6

## 8.8) Calcolo e plot del dNBR classificato secondo lo standard USGS, nel periodo dell'incendio del 2017, considerando come AOI la Piana del Fucino

In [ ]:
#Definisco l'AOI attraverso le coordinate del poligono che racchiude l'area di studio
AOI_fucino = ee.Geometry.Polygon(
    [[13.504274, 42.061546],
    [13.452099, 42.036051],
    [13.432877, 41.990136],
    [13.485052, 41.955422],
    [13.533107, 41.95338],
    [13.627845, 41.944187],
    [13.659424, 41.966655],
    [13.619607, 42.007486],
    [13.564686, 42.051349],
    [13.504274, 42.061546]])

#Creo una ImageCollection con i dati di Sentinel 2 (S2) per pre-incendio e post-incendio 2017
collection_prefire_2017_fucino = ee.ImageCollection("COPERNICUS/S2_SR") \
.filterDate(start_prefire_2023, finish_prefire_2023) \
.filterBounds(AOI_fucino) \
.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',1)); 


collection_endfire_2017_fucino = ee.ImageCollection("COPERNICUS/S2_SR") \
.filterDate(start_endfire_2023, finish_endfire_2023) \
.filterBounds(AOI_fucino) \
.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',1)); 

#Prendo la mediana delle variabili ImageCollection. Essa rappresenta una statistica di tutti i passaggi. 
#Al termine di questa operazione ottengo le immagini image_prefire_2017_fucino e image_endfire_fucino
#NB: avrei potuto anche prendere direttamente una sola immagine, non nuvolosa, senza passare per la statistica dei passaggi 

#Incendio 2017
image_prefire_2017_fucino = ee.Image(collection_prefire_2017_fucino.median())
image_endfire_2017_fucino = ee.Image(collection_endfire_2017_fucino.median())

#Calcolo del Normalized Burn Ratio (NBR) pre e post incendio 2017
preNBR_2017_fucino = image_prefire_2017_fucino.normalizedDifference(['B8', 'B12'])
endNBR_2017_fucino = image_endfire_2017_fucino.normalizedDifference(['B8', 'B12'])

#Calcolo il differenced NBR (dNBR) per l'incendio del 2017
#dNBR = PrefireNBR - EndfireNBR
dNBR_unscaled_2017_fucino = preNBR_2017_fucino.subtract(endNBR_2017_fucino)
#Lo scalo per poi riferirmi agli standard USGS
dNBR_2017_fucino = dNBR_unscaled_2017_fucino.multiply(1000)

#Definisco una nuova mappa interattiva, plotto le immagini e aggiungo la legenda
centro_fucino=[41.999117, 13.539972]
mappa7 = geemap.Map(center=centro_fucino, zoom=13)
#Aggiungo l'immagine come layer caratterizzata dai valori di dNBR classificati secondo lo standard USGS, nel periodo 
#dell'evento del Morrone del 2017, ma calcolato per l'AOI della Piana del Fucino 
mappa7.addLayer(dNBR_2017_fucino.sldStyle(sld_intervals).clip(AOI_fucino), {}, "dNBR classificato Piana del Fucino 2017")
mappa7.add_legend(
    title='dNBR standard USGS', legend_dict=legend_dict, draggable=False, style=style
)
mappa7